<a href="https://colab.research.google.com/github/ssarkar551/Dance-Forms/blob/main/Dance_Form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile

In [ ]:
local_zip='/content/0664343c9a8f11ea.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
os.mkdir("/content/train")
#!cp normal ~/.train
#!chmod 600 ~/.train/normal

In [ ]:
import pandas as pd
train=pd.read_csv('/content/dataset/train.csv')

In [ ]:
train.head()

In [ ]:
test=pd.read_csv('/content/dataset/test.csv')
test.head()

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.preprocessing import image

In [ ]:
train_images=os.listdir('/content/dataset/train')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Pipelining the images

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
            ('cat', categorical_transformer, train['target'])
    ])


In [ ]:
import tensorflow as tf


Building the model

In [ ]:
model=tf.keras.models.Sequential([
                                  tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPooling2D(2,2),
                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(512,activation='relu'),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
import numpy as np
y = np.array(train['target'])

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_images, y, random_state=42, test_size=0.1)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=40,
    zoom_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

In [ ]:
train_generator=train_datagen.flow_from_dataframe(train_images,
                                                  target_size=(150,150),
                                                  class_mode='categorical',
                                                  batch_size=20,
                                                  )

In [ ]:
model.fit(X_train, train[''], steps_per_epoch=50, epochs=25)